# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 4 2022 12:28PM,243462,20,8359328,"Exact-Rx, Inc.",Released
1,Aug 4 2022 12:28PM,243462,20,8359329,"Exact-Rx, Inc.",Released
2,Aug 4 2022 12:28PM,243462,20,8359330,"Exact-Rx, Inc.",Released
3,Aug 4 2022 12:28PM,243462,20,8359331,"Exact-Rx, Inc.",Released
4,Aug 4 2022 12:24PM,243461,20,ALUR544291587,Alumier Labs Inc.,Released
5,Aug 4 2022 12:24PM,243461,20,ALU0007453,Alumier Labs Inc.,Released
6,Aug 4 2022 12:24PM,243461,20,ALUR043334783,Alumier Labs Inc.,Released
7,Aug 4 2022 12:24PM,243461,20,ALUR786060791,Alumier Labs Inc.,Released
8,Aug 4 2022 12:24PM,243461,20,ALUR056810571,Alumier Labs Inc.,Released
9,Aug 4 2022 12:24PM,243461,20,ALUR951589780,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,243458,Released,38
21,243459,Released,33
22,243460,Released,4
23,243461,Released,19
24,243462,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243458,NaN,38.0
243459,NaN,33.0
243460,NaN,4.0
243461,NaN,19.0
243462,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243414,0.0,1.0
243415,0.0,1.0
243417,6.0,3.0
243435,13.0,2.0
243436,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243414,0,1
243415,0,1
243417,6,3
243435,13,2
243436,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243414,0,1
1,243415,0,1
2,243417,6,3
3,243435,13,2
4,243436,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243414,,1
1,243415,,1
2,243417,6,3
3,243435,13,2
4,243436,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 4 2022 12:28PM,243462,20,"Exact-Rx, Inc."
4,Aug 4 2022 12:24PM,243461,20,Alumier Labs Inc.
23,Aug 4 2022 12:11PM,243458,10,ISDIN Corporation
61,Aug 4 2022 12:06PM,243460,10,"Methapharm, Inc."
65,Aug 4 2022 12:05PM,243459,10,"Methapharm, Inc."
98,Aug 4 2022 11:53AM,243457,20,Alumier Labs Inc.
116,Aug 4 2022 11:40AM,243454,20,Alumier Labs Inc.
117,Aug 4 2022 11:38AM,243453,20,Alumier Labs Inc.
132,Aug 4 2022 11:36AM,243452,20,Alumier Labs Inc.
143,Aug 4 2022 11:35AM,243456,10,Beach Products Inc


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 4 2022 12:28PM,243462,20,"Exact-Rx, Inc.",,4
1,Aug 4 2022 12:24PM,243461,20,Alumier Labs Inc.,,19
2,Aug 4 2022 12:11PM,243458,10,ISDIN Corporation,,38
3,Aug 4 2022 12:06PM,243460,10,"Methapharm, Inc.",,4
4,Aug 4 2022 12:05PM,243459,10,"Methapharm, Inc.",,33
5,Aug 4 2022 11:53AM,243457,20,Alumier Labs Inc.,,18
6,Aug 4 2022 11:40AM,243454,20,Alumier Labs Inc.,,1
7,Aug 4 2022 11:38AM,243453,20,Alumier Labs Inc.,,15
8,Aug 4 2022 11:36AM,243452,20,Alumier Labs Inc.,,11
9,Aug 4 2022 11:35AM,243456,10,Beach Products Inc,,13


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 12:28PM,243462,20,"Exact-Rx, Inc.",4,
1,Aug 4 2022 12:24PM,243461,20,Alumier Labs Inc.,19,
2,Aug 4 2022 12:11PM,243458,10,ISDIN Corporation,38,
3,Aug 4 2022 12:06PM,243460,10,"Methapharm, Inc.",4,
4,Aug 4 2022 12:05PM,243459,10,"Methapharm, Inc.",33,
5,Aug 4 2022 11:53AM,243457,20,Alumier Labs Inc.,18,
6,Aug 4 2022 11:40AM,243454,20,Alumier Labs Inc.,1,
7,Aug 4 2022 11:38AM,243453,20,Alumier Labs Inc.,15,
8,Aug 4 2022 11:36AM,243452,20,Alumier Labs Inc.,11,
9,Aug 4 2022 11:35AM,243456,10,Beach Products Inc,13,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 12:28PM,243462,20,"Exact-Rx, Inc.",4,
1,Aug 4 2022 12:24PM,243461,20,Alumier Labs Inc.,19,
2,Aug 4 2022 12:11PM,243458,10,ISDIN Corporation,38,
3,Aug 4 2022 12:06PM,243460,10,"Methapharm, Inc.",4,
4,Aug 4 2022 12:05PM,243459,10,"Methapharm, Inc.",33,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 12:28PM,243462,20,"Exact-Rx, Inc.",4.0,NaN
1,Aug 4 2022 12:24PM,243461,20,Alumier Labs Inc.,19.0,NaN
2,Aug 4 2022 12:11PM,243458,10,ISDIN Corporation,38.0,NaN
3,Aug 4 2022 12:06PM,243460,10,"Methapharm, Inc.",4.0,NaN
4,Aug 4 2022 12:05PM,243459,10,"Methapharm, Inc.",33.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 12:28PM,243462,20,"Exact-Rx, Inc.",4.0,0.0
1,Aug 4 2022 12:24PM,243461,20,Alumier Labs Inc.,19.0,0.0
2,Aug 4 2022 12:11PM,243458,10,ISDIN Corporation,38.0,0.0
3,Aug 4 2022 12:06PM,243460,10,"Methapharm, Inc.",4.0,0.0
4,Aug 4 2022 12:05PM,243459,10,"Methapharm, Inc.",33.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2434462,96.0,49.0
12,243440,0.0,1.0
19,730313,11.0,7.0
20,1460739,68.0,0.0
21,243414,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2434462,96.0,49.0
1,12,243440,0.0,1.0
2,19,730313,11.0,7.0
3,20,1460739,68.0,0.0
4,21,243414,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,96.0,49.0
1,12,0.0,1.0
2,19,11.0,7.0
3,20,68.0,0.0
4,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,96.0
1,12,Released,0.0
2,19,Released,11.0
3,20,Released,68.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20,21
Status,,,,,
Executing,49.0,1.0,7.0,0.0,0.0
Released,96.0,0.0,11.0,68.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20,21
0,Executing,49.0,1.0,7.0,0.0,0.0
1,Released,96.0,0.0,11.0,68.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20,21
0,Executing,49.0,1.0,7.0,0.0,0.0
1,Released,96.0,0.0,11.0,68.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()